# Creating and manipulating a `Spectrum`

In [8]:
# Imports
from SEDkit import spectrum, synphot
import astropy.units as q
import astropy.table as at
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

Let's create a new `Spectrum` object by passing it arrays with astropy units

In [3]:
# Get the data from file
file = resource_filename('SEDkit', 'data/M0V_spectrum.txt')
wave, flux, unc = np.genfromtxt(file, unpack=True)
spec = spectrum.Spectrum(wave*q.um, flux*q.W/q.m**2/q.um, unc*q.W/q.m**2/q.um)

Warning, 765 of 7119 bins contained negative fluxes; they have been set to zero.


We can calculate a synthetic flux or magnitude in a given band by passing it a bandpass

In [5]:
jband = synphot.Bandpass('2MASS.J')
print('Jflux =',spec.synthetic_flux(jband))
print('Jmag =',spec.synthetic_magnitude(jband))

Jflux = (<Quantity 7.86148695630033e-12 W / (m2 um)>, <Quantity 1.981354543406238e-12 W / (m2 um)>)
Jmag = (6.4936122335790136, 0.27364140832341316)


A spectrum can also be bormalized to given magnitude in a given band

In [7]:
print('Old Jflx =',spec.synthetic_flux(jband))
normed = spec.renormalize(16.332, jband)
print('New Jflux =',normed.synthetic_flux(jband))

Old Jflx = (<Quantity 7.86148695630033e-12 W / (m2 um)>, <Quantity 1.981354543406238e-12 W / (m2 um)>)
New Jflux = (<Quantity 1.0900739577044195e-18 W / (m2 um)>, <Quantity 2.74734665433179e-19 W / (m2 um)>)
